In [2]:
import numpy as np
import pandas as pd

pr = pd.read_csv('data/cargo-pagerank.csv', delimiter=',');

In [8]:
#Get downloads data from crates
import requests

data = requests.get('https://crates.io/api/v1/crates?page=1&per_page=100&sort=downloads').json()['crates']
df = pd.DataFrame.from_dict(data)
df.head()

,badges,categories,created_at,description,documentation,downloads,exact_match,homepage,id,keywords,links,max_version,name,recent_downloads,repository,updated_at,versions
0,"[{'attributes': {'branch': None, 'repository':...",None,2015-01-15T20:22:13.100871+00:00,Raw FFI bindings to platform libraries like li...,http://doc.rust-lang.org/libc,14739035,False,https://github.com/rust-lang/libc,libc,None,"{'versions': '/api/v1/crates/libc/versions', '...",0.2.55,libc,1984106,https://github.com/rust-lang/libc,2019-05-17T06:42:13.034356+00:00,None
1,"[{'attributes': {'branch': None, 'repository':...",None,2015-02-03T06:17:14.147783+00:00,Random number generators and other randomness ...,https://rust-random.github.io/rand,14712942,False,https://crates.io/crates/rand,rand,None,"{'versions': '/api/v1/crates/rand/versions', '...",0.6.5,rand,3124033,https://github.com/rust-random/rand,2019-01-28T09:56:57.788327+00:00,None
2,"[{'attributes': {'branch': None, 'repository':...",None,2015-01-15T08:35:24.901276+00:00,A macro to generate structures which behave li...,https://docs.rs/bitflags,13782257,False,https://github.com/bitflags/bitflags,bitflags,None,{'versions': '/api/v1/crates/bitflags/versions...,1.0.4,bitflags,1925556,https://github.com/bitflags/bitflags,2019-05-02T06:02:56.939362+00:00,None
3,"[{'attributes': {'branch': None, 'id': None, '...",None,2014-11-23T01:18:10.993977+00:00,A macro for declaring lazily evaluated statics...,https://docs.rs/lazy_static,11795959,False,None,lazy_static,None,{'versions': '/api/v1/crates/lazy_static/versi...,1.3.0,lazy_static,2005020,https://github.com/rust-lang-nursery/lazy-stat...,2019-02-27T00:02:09.802332+00:00,None
4,"[{'attributes': {'branch': None, 'id': None, '...",None,2014-12-13T22:10:19.944550+00:00,A lightweight logging facade for Rust\n,https://docs.rs/log,11756373,False,https://github.com/rust-lang/log,log,None,"{'versions': '/api/v1/crates/log/versions', 'o...",0.4.6,log,1905940,https://github.com/rust-lang/log,2018-10-27T19:06:10.563576+00:00,None


In [9]:
len(df)

100

In [29]:
#Compose dependency metadata data frame
df_downloads = df[['id', 'downloads']].copy()
#Rename Columns
df_downloads = df_downloads.rename(columns={'id': 'Name', 'downloads': 'Downloads'})
df_downloads.head()

,Name,Downloads
0,libc,14739035
1,rand,14712942
2,bitflags,13782257
3,lazy_static,11795959
4,log,11756373


In [32]:
#Load Pageranks and merge with Downloads data
df_deps = pd.read_csv('data/cargo-pagerank.csv', delimiter=',');


df_deps_downloads = pd.merge(df_deps, df_downloads, on='Name')
df_deps_downloads.head()

,ID,Name,page_rank,Downloads
0,1369785,num-traits,0.059415,8760484
1,428702,rand,0.035209,14712942
2,31533,winapi,0.033656,8705299
3,31384,serde,0.026141,10746355
4,31115,libc,0.022697,14739035


In [39]:
#Add Ranks for Pagerank and Downloads to data frame
df_deps_downloads['PR_Rank'] = df_deps_downloads.page_rank.rank(ascending= False)
df_deps_downloads['DL_Rank'] = df_deps_downloads.Downloads.rank(ascending= False)
df_deps_downloads.head()

,ID,Name,page_rank,Downloads,PR_Rank,DL_Rank
0,1369785,num-traits,0.059415,8760484,1.0,12.0
1,428702,rand,0.035209,14712942,2.0,2.0
2,31533,winapi,0.033656,8705299,3.0,13.0
3,31384,serde,0.026141,10746355,4.0,6.0
4,31115,libc,0.022697,14739035,5.0,1.0


In [40]:
#Write to osrank analysis csv
df_deps_downloads.to_csv('data/cargo-osrank-analysis.csv', index= False)